# TODO
    - break out the fuctions into a separate file.
    - incorporate seeds into the two simulation steps as well as the sampling steps.
    - batch simulation steps 
    - figure out ancestral ne, and what scaling the mutation rate means in practice
    - figure out how to simulate unlinked loci (just run msprime many, many times??)
    - check the controlled mating is doing the expected
    - batch the analysis


In [2]:
import numpy as np
import pandas as pd
import itertools
import collections
import glob
import random
from random import shuffle

In [3]:
import msprime

In [4]:
#from pylab import rcParams
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [13]:
def make_rec_map(n_chrom, len_chrom, recombination_rate, finite_sites):
    n_chrom = int(n_chrom)
    len_chrom = int(len_chrom)
    A = [[x, x+1] for x in range(int(len_chrom), int(len_chrom)*(n_chrom+1), int(len_chrom))]
    rm_pos = [0] + list(itertools.chain.from_iterable(A))
    B = [recombination_rate, 0.5] * n_chrom
    rec_rates = B + [0]
    rec_map = msprime.RecombinationMap(rm_pos, rec_rates, num_loci = finite_sites )
    return(rec_map)

# Start

In [14]:
# coalescent simulation
batchname = 'feb18'
!mkdir {'/home/ryan/simNe/share/{}'.format(batchname)}

chrom_len = 5e7
mutation_rate = 1e-8
recombination_rate = 1e-8
Ne_ancestral = 2000
Ne_switchdate = 100
coalescent_seed = 1001
forward_seed = 1001
repilcates = 500

mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18’: File exists


In [15]:
n_chroms = [4, 16, 64]
diploid_Nes = [50, 200, 800, 3200]

In [16]:
for n_chrom in n_chroms:
    for diploid_Ne in diploid_Nes:
        print('working on n_chrom = {}, diploid_Ne = {}'.format(n_chrom, diploid_Ne))
        # make directories and write the params file
        !mkdir {'/home/ryan/simNe/share/{}/Ne-{}_Chr-{}'.format(batchname, diploid_Ne, n_chrom)}
        with open('/home/ryan/simNe/share/{}/Ne-{}_Chr-{}/params.txt'.format(batchname, diploid_Ne, n_chrom), 'w') as OUTFILE:
            OUTFILE.write("n_chrom\t{}\n".format(n_chrom))
            OUTFILE.write("chrom_len\t{}\n".format(chrom_len))
            OUTFILE.write("diploid_Ne\t{}\n".format(diploid_Ne))
            OUTFILE.write("mutation_rate\t{}\n".format(mutation_rate))
            OUTFILE.write("batchname\t{}\n".format(batchname))
            OUTFILE.write("repilcates\t{}\n".format(repilcates))
            OUTFILE.write("Ne_ancestral\t{}\n".format(Ne_ancestral))
            OUTFILE.write("Ne_switchdate\t{}\n".format(Ne_switchdate))
            OUTFILE.write("coalescent_seed\t{}\n".format(coalescent_seed))
            OUTFILE.write("forward_seed\t{}\n".format(forward_seed))

working on n_chrom = 4, diploid_Ne = 50
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-50_Chr-4’: File exists
working on n_chrom = 4, diploid_Ne = 200
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-200_Chr-4’: File exists
working on n_chrom = 4, diploid_Ne = 800
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-800_Chr-4’: File exists
working on n_chrom = 4, diploid_Ne = 3200
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-3200_Chr-4’: File exists
working on n_chrom = 16, diploid_Ne = 50
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-50_Chr-16’: File exists
working on n_chrom = 16, diploid_Ne = 200
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-200_Chr-16’: File exists
working on n_chrom = 16, diploid_Ne = 800
mkdir: cannot create directory ‘/home/ryan/simNe/share/feb18/Ne-800_Chr-16’: File exists
working on n_chrom = 16, diploid_Ne = 3200
mkdir: cannot create directory ‘/home/ryan/simNe/s

In [17]:
demographic_events = [
        msprime.PopulationParametersChange(
            time=Ne_switchdate, initial_size=Ne_ancestral)
]

In [18]:
for n_chrom in n_chroms:
    for diploid_Ne in diploid_Nes:
        finite_sites = int(n_chrom * chrom_len + n_chrom - 1)//10
        rm = make_rec_map(n_chrom, chrom_len, recombination_rate, finite_sites = finite_sites )
        sim_data = msprime.simulate(
                    Ne = diploid_Ne*2,
                    mutation_rate=mutation_rate,
                    recombination_map = rm,
                    #recombination_map = None,
                    #recombination_rate = 1e-8,
                    #length = 160,
                    #population_configurations = None,
                    sample_size = diploid_Ne*2,
                    demographic_events = demographic_events,
                    )
        sim_data.dump('/home/ryan/simNe/share/{}/Ne-{}_Chr-{}/coalescent.hdf5'.format(batchname, diploid_Ne, n_chrom), zlib_compression=False)

KeyboardInterrupt: 

In [ ]:
# rerun with fewer sites

In [ ]:
n_chroms = [64]
for n_chrom in n_chroms:
    for diploid_Ne in diploid_Nes:
        finite_sites = int(n_chrom * chrom_len + n_chrom - 1)//100
        rm = make_rec_map(n_chrom, chrom_len, recombination_rate, finite_sites = finite_sites )
        sim_data = msprime.simulate(
                    Ne = diploid_Ne*2,
                    mutation_rate=mutation_rate,
                    recombination_map = rm,
                    #recombination_map = None,
                    #recombination_rate = 1e-8,
                    #length = 160,
                    #population_configurations = None,
                    sample_size = diploid_Ne*2,
                    demographic_events = demographic_events,
                    )
        sim_data.dump('/home/ryan/simNe/share/{}/Ne-{}_Chr-{}/coalescent.hdf5'.format(batchname, diploid_Ne, n_chrom), zlib_compression=False)

In [ ]:
assert False

In [ ]:
for n_chrom in n_chroms:
    for diploid_Ne in diploid_Nes:
        sim_data = simNe_sim.sim_chrom(
            n_chrom = n_chrom,
            chrom_len = int(chrom_len),
            recombination_rate = 1.0/chrom_len,
            mutation_rate = mutation_rate, # could be scaled by a multiplier to simply get fewer loci
            sample_size = diploid_Ne*2, # haploid
            Ne_ancestral = Ne_ancestral,
            Ne_recent = diploid_Ne*2,
            Ne_switchdate = 100, # fixed switch 
            # random_seed = coalescent_seed,
            print_history = False)
        sim_data.dump('/home/ryan/simNe/share/{}/Ne-{}_Chr-{}/coalescent.hdf5'.format(batchname, diploid_Ne, n_chrom), zlib_compression=False)

In [ ]:

        # do coalescent simulation 
        sim_data = simNe_sim.sim_chrom(
            n_chrom = n_chrom,
            chrom_len = int(chrom_len),
            recombination_rate = 1.0/chrom_len,
            mutation_rate = mutation_rate, # could be scaled by a multiplier to simply get fewer loci
            sample_size = diploid_Ne*2, # haploid
            Ne_ancestral = Ne_ancestral,
            Ne_recent = diploid_Ne*2,
            Ne_switchdate = 100, # fixed switch 
            # random_seed = coalescent_seed,
            print_history = False)
        
        # record the coalescent simualtion
        sim_data.dump('/home/ryan/simNe/share/{}/Ne-{}_Chr-{}/coalescent.hdf5'.format(batchname, diploid_Ne, n_chrom), zlib_compression=False)

In [ ]:
sim_data = simNe_sim.sim_chrom(
            n_chrom = 4,
            chrom_len = chrom_len,
            recombination_rate = mutation_rate,
            mutation_rate = mutation_rate, # could be scaled by a multiplier to simply get fewer loci
            sample_size = diploid_Ne*2, # haploid
            Ne_ancestral = Ne_ancestral,
            Ne_recent = diploid_Ne*2,
            Ne_switchdate = 100, # fixed switch 
            #random_seed = coalescent_seed,
            print_history = False)